In [12]:
from gutenberg.acquire import load_etext
from gutenberg.cleanup import strip_headers
import pandas as pd
import numpy as np
import spacy
from collections import Counter

In [13]:
#read in
ch_dir ='swiss_history_books/Der_Geschichten_schweizerischer_Eidgenos.txt'
fr_dir = 'swiss_history_books/Histoire_de_la_Confédération_Suisse-1.txt'
with open(ch_dir) as f:
    schweizer_geschichte = f.read()
    
with open(fr_dir) as f:
    confederation_suisse = f.read()


In [14]:
# preprocessing
pos_to_keep = ['ADJ', 'ADV', 'NOUN', 'PROPN', 'VERB']

def lematization_pos_tagging(tagging_language_funtion, text):
    doc = tagging_language_funtion(text)
    lemmatized = [x.lemma_ for x in doc if x.pos_ in pos_to_keep and x.is_stop != True]
    return lemmatized

#load spacy corpuses
nlp_de = spacy.load('de_core_news_md')
nlp_fr = spacy.load('fr_core_news_sm')

In [15]:
#lemmatize german and french version
schweizer_geschichte = schweizer_geschichte.replace('ſ', 's')#replace ſ with s
schweizer_geschichte_lematized = lematization_pos_tagging(nlp_de, schweizer_geschichte)
schweizer_tf = Counter(schweizer_geschichte_lematized)
confederation_suisse_lemmatized = lematization_pos_tagging(nlp_fr, confederation_suisse)
confederation_tf = Counter(confederation_suisse_lemmatized)



In [55]:
#pre processing with lematization

#print(schweizer_tf)
#print((confederation_tf))
schweizer_tf_top_100 = [(el, schweizer_tf[el])   for  el in schweizer_tf]
schweizer_tf_top_100 = sorted(schweizer_tf_top_100, key=lambda tup: tup[1], reverse=True)
confederation_tf_top_100 = [(el, confederation_tf[el])  for el in confederation_tf]
confederation_tf_top_100 = sorted(confederation_tf_top_100, key=lambda tup: tup[1], reverse=True)


print(schweizer_tf_top_100[:100])
print(confederation_tf_top_100[:100])




[('Regierung', 476), ('Kanton', 337), ('Schweiz', 291), ('Frankreich', 221), ('Truppe', 203), ('Bern', 198), ('Partei', 183), ('lassen', 175), ('Consul', 170), ('ward', 169), ('französisch', 167), ('Land', 163), ('Mann', 157), ('Verfassung', 147), ('fein', 139), ('alt', 138), ('Minister', 136), ('helvetischen', 134), ('Volk', 123), ('geben', 121), ('finden', 119), ('de', 118), ('Verninac', 113), ('Stadt', 107), ('erhalten', 103), ('Ver', 101), ('Helvetien', 94), ('Mitglied', 94), ('halten', 93), ('Min', 92), ('Paris', 91), ('Behörde', 90), ('bringen', 89), ('Wallis', 88), ('Abgeordnete', 85), ('Art', 84), ('Republik', 83), ('stellen', 83), ('Bürger', 83), ('Interesse', 82), ('bleiben', 81), ('Senat', 81), ('verlangen', 79), ('Seite', 79), ('erklären', 79), ('August', 75), ('Consuls', 73), ('Staat', 73), ('Vollz', 72), ('scheinen', 72), ('ziehen', 72), ('helvetische', 72), ('1.', 71), ('Reinhard', 71), ('Beschluß', 71), ('General', 70), ('Mai', 70), ('Gewalt', 70), ('öffentlich', 70), (

In [11]:
#comparison corpus german
from tei_reader import TeiReader
reader = TeiReader()




import os
from xml.dom import minidom
import xml.etree.ElementTree as ET
dir_de_ref = 'swiss_history_books/reference_corpus_german/dta_kernkorpus_1700-1799_2020-07-20/'
dir_de_lemmatized = 'swiss_history_books/reference_corpus_german/dta_kernkorpus_1700-1799_2020-07-20_lemmatized/'

text_corpus_de = []
for files in os.listdir(dir_de_ref)[:50]:
    mydoc = minidom.parse(dir_de_ref+files)
    corpora = reader.read_file(dir_de_ref+files)
    
    ref_text = corpora.text.replace('ſ', 's')#replace ſ with s
    ref_text_cleaned = ' '.join([el for el in ref_text.split() if len(el) > 1])
    
    ref_text_lematized = lematization_pos_tagging(nlp_de, ref_text_cleaned[:1000000])#rate limit at 1 million limit texts
    text_corpus_de.append(ref_text_lematized)
    
    with open(dir_de_lemmatized+files[:-4]+".txt", "w") as text_file:
        text_file.write(' '.join(ref_text_lematized))
    
    
    



In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

text_corpus_de = []
for path in os.listdir(dir_de_lemmatized):
    with open(dir_de_lemmatized+path) as f:
         text_corpus_de.append(f.read())

text_corpus_de.append(' '.join(schweizer_geschichte_lematized))


vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(text_corpus_de)




In [48]:
import numpy as np
print(np.shape(X))

#.get_feature_names()
schweizer_geschichte_tfidf = pd.DataFrame(X[-1].toarray().T)
schweizer_geschichte_tfidf.index = vectorizer.get_feature_names()
schweizer_geschichte_tfidf = schweizer_geschichte_tfidf.sort_values(by = 0, ascending=False)

schweizer_geschichte_tfidf[:50]



(51, 299506)


,0
kanton,0.381750
schweiz,0.246899
regierung,0.227902
partei,0.192207
truppe,0.177800
französisch,0.175583
consul,0.148897
helvetischen,0.140259
verninac,0.140229
frankreich,0.136417
